## UAIR Application - Work Sample
### Anonymized Student Data obtained from [Open University Learning Analytics](https://analyse.kmi.open.ac.uk/open_dataset) 
#### (Database schema and descriptions can be found at that site)

In [1]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3

import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
# for in-memory database

# conn = sqlite3.connect(':memory:')
# c = conn.cursor()

## I. Obtain Data

### Test using pandas to convert csv files to SQL

In [3]:
courses_df = pd.read_csv('data/courses.csv')
courses_df.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [5]:
courses_df.to_sql('courses', conn, "'if_exists'='append'", index=False)

In [6]:
c.execute('SELECT * FROM courses').fetchall()

[('AAA', '2013J', 268),
 ('AAA', '2014J', 269),
 ('BBB', '2013J', 268),
 ('BBB', '2014J', 262),
 ('BBB', '2013B', 240),
 ('BBB', '2014B', 234),
 ('CCC', '2014J', 269),
 ('CCC', '2014B', 241),
 ('DDD', '2013J', 261),
 ('DDD', '2014J', 262),
 ('DDD', '2013B', 240),
 ('DDD', '2014B', 241),
 ('EEE', '2013J', 268),
 ('EEE', '2014J', 269),
 ('EEE', '2014B', 241),
 ('FFF', '2013J', 268),
 ('FFF', '2014J', 269),
 ('FFF', '2013B', 240),
 ('FFF', '2014B', 241),
 ('GGG', '2013J', 261),
 ('GGG', '2014J', 269),
 ('GGG', '2014B', 241)]

### Test with new database

In [9]:
c.close()

In [10]:
conn = sqlite3.connect('university.db')
c = conn.cursor()

In [11]:
assessments_df = pd.read_csv('data/assessments.csv')
assessments_df.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [12]:
assessments_df.to_sql('assessments', conn, "'if_exists'='append'", index=False)

In [13]:
# Check to see if assessments table present within university.db

c.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()

[('assessments',)]

In [15]:
# manually read-in remaining csv files, I know this is ugly

courses_df = pd.read_csv('data/courses.csv')
student_assessment_df = pd.read_csv('data/studentAssessment.csv')
student_info_df = pd.read_csv('data/studentInfo.csv')
student_registration_df = pd.read_csv('data/studentRegistration.csv')
student_vle_df = pd.read_csv('data/studentVle.csv')
vle_df = pd.read_csv('data/vle.csv')

#### Function to add dataframe to SQL database

In [16]:
def add_df_to_db(df, table_name):
    
    df.to_sql(table_name, conn, "'if_exists'='append'", index=False)

In [24]:
# okay, figuring out the for-loops is going to take longer than doing it manually, here is some more ugliness to enjoy

add_df_to_db(courses_df, 'courses')
add_df_to_db(student_assessment_df, 'student_assessments')
add_df_to_db(student_info_df, 'student_info')
add_df_to_db(student_registration_df, 'student_registration')
add_df_to_db(student_vle_df, 'student_vle')
add_df_to_db(vle_df, 'vle')

In [25]:
# check that all tables are in db

c.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()

[('assessments',),
 ('courses',),
 ('student_assessments',),
 ('student_info',),
 ('student_registration',),
 ('student_vle',),
 ('vle',)]

In [26]:
# check that values are actually in the tables!

c.execute("SELECT * FROM student_assessments;").fetchall()

[(1752, 11391, 18, 0, 78.0),
 (1752, 28400, 22, 0, 70.0),
 (1752, 31604, 17, 0, 72.0),
 (1752, 32885, 26, 0, 69.0),
 (1752, 38053, 19, 0, 79.0),
 (1752, 45462, 20, 0, 70.0),
 (1752, 45642, 18, 0, 72.0),
 (1752, 52130, 19, 0, 72.0),
 (1752, 53025, 9, 0, 71.0),
 (1752, 57506, 18, 0, 68.0),
 (1752, 58873, 19, 0, 73.0),
 (1752, 59185, 18, 0, 67.0),
 (1752, 62155, 17, 0, 73.0),
 (1752, 63400, 19, 0, 83.0),
 (1752, 65002, 17, 0, 66.0),
 (1752, 70464, 19, 0, 59.0),
 (1752, 71361, 19, 0, 82.0),
 (1752, 74372, 22, 0, 60.0),
 (1752, 75091, 18, 0, 67.0),
 (1752, 77367, 18, 0, 73.0),
 (1752, 91265, 21, 0, 75.0),
 (1752, 94961, 17, 0, 74.0),
 (1752, 98094, 18, 0, 62.0),
 (1752, 100893, 17, 0, 63.0),
 (1752, 101781, 16, 0, 84.0),
 (1752, 102806, 19, 0, 80.0),
 (1752, 102952, 19, 0, 76.0),
 (1752, 104476, 30, 0, 85.0),
 (1752, 106247, 32, 0, 67.0),
 (1752, 106577, 18, 0, 57.0),
 (1752, 110175, 18, 0, 66.0),
 (1752, 111717, 10, 0, 80.0),
 (1752, 113295, 19, 0, 81.0),
 (1752, 114017, 18, 0, 66.0),
 (17

## II. Exploratory Data Analysis

### Student Info - Demographic Data about Students and Their Results

Columns that are not self-explanatory

* imd_band = Index of Multiple Deprivation, where the student lived during the module presentation (score used in the United Kingdom to assess a region based on income, employment, health deprivation and disability, education skills and training, barriers to housing and services, crime, and living environment
* studied_credits = total number of credits for the modules the student is currently studying

In [27]:
student_info_df.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [29]:
c.execute('SELECT DISTINCT imd_band from student_info;').fetchall()

[('90-100%',),
 ('20-30%',),
 ('30-40%',),
 ('50-60%',),
 ('80-90%',),
 ('70-80%',),
 (None,),
 ('60-70%',),
 ('40-50%',),
 ('10-20',),
 ('0-10%',)]

In [31]:
student_info_df.isnull().sum()

code_module                0
code_presentation          0
id_student                 0
gender                     0
region                     0
highest_education          0
imd_band                1111
age_band                   0
num_of_prev_attempts       0
studied_credits            0
disability                 0
final_result               0
dtype: int64

In [33]:
student_info_df.shape

(32593, 12)

In [34]:
student_info_df.imd_band.value_counts()

20-30%     3654
30-40%     3539
10-20      3516
0-10%      3311
40-50%     3256
50-60%     3124
60-70%     2905
70-80%     2879
80-90%     2762
90-100%    2536
Name: imd_band, dtype: int64

In [37]:
student_info_df.final_result.value_counts()

Pass           12361
Withdrawn      10156
Fail            7052
Distinction     3024
Name: final_result, dtype: int64

#### Investigate this interesting IMD index

In [57]:
c.execute("""
        SELECT imd_band, COUNT(final_result) AS 'pass'
        FROM student_info
        WHERE final_result = 'Pass'
        UNION
        SELECT imd_band, COUNT(final_result) AS 'withdrawn'
        FROM student_info
        WHERE final_result = 'Withdrawn';""").fetchall()

[('30-40%', 10156), ('90-100%', 12361)]

### Non-Pandas Way

In [8]:
# this works, but the pandas option is way better because you don't have to manually input column names

# open in-memory database 

# con = sqlite3.connect(":memory:")
# cur = con.cursor()
# cur.execute("CREATE TABLE courses (code_module, code_presentation, module_presentation_length);")

# with open('data/courses.csv','rb') as fin:
#     dr = csv.DictReader(fin)
#     # dicts = ({'code_module': line[0], 'code_presentation': line[1], 'module_presentation_length': line[2]} for line in dr)
#     to_db = ((i['code_module'], i['code_presentation'], i['module_presentation_length']) for i in dicts)

# cur.executemany("INSERT INTO courses (code_module, code_presentation, module_presentation_length) VALUES (?, ?, ?);", to_db)
# con.commit()

# import csv, sqlite3

# con = sqlite3.connect(":memory:")
# cur = con.cursor()
# cur.execute("CREATE TABLE t (col1, col2);") # use your column names here

# with open('data.csv','rb') as fin: # `with` statement available in 2.5+
#     # csv.DictReader uses first line in file for column headings by default
#     dr = csv.DictReader(fin) # comma is default delimiter
#     to_db = [(i['col1'], i['col2']) for i in dr]

# cur.executemany("INSERT INTO t (col1, col2) VALUES (?, ?);", to_db)
# con.commit()
# con.close()

In [6]:
# con.close()

In [7]:
# this code shows that the table courses has been created
# cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()